# Dual Rail Entanglement Heralding

Here we implement a Dual Rail Entanglement Heralding (DREH) protocol 

In [109]:
import numpy as np
import time
import netsquid as ns
from netsquid.qubits import qubitapi as qapi

from netsquid.components import QSource, Clock
from netsquid.components.qsource import SourceStatus
from netsquid.qubits.state_sampler import StateSampler
from netsquid.qubits.ketstates import b00
from netsquid.qubits.operators import H, CX

from netsquid.nodes import Node
from netsquid.components import Component, QuantumChannel, ClassicalChannel, QuantumProcessor, PhysicalInstruction, Switch, QuantumDetector
from netsquid.components import instructions as instr
from netsquid.components.qprogram import QuantumProgram
from netsquid.components.models import FibreDelayModel, FibreLossModel, DepolarNoiseModel
from netsquid.examples.repeater_chain import FibreDepolarizeModel

## Debugging functions

In [110]:
def print_source_output_message(message):
    # Print qubit, emission time and emission delay.
    print("Qubits {} emitted at t={} with an emission delay of {} nano seconds"
         .format(message.items, message.meta["emission_time"], message.meta["emission_delay"]))

def debug_output(message):
    name = message.items[0].name
    if 'source_a' in name:
        print(f"Received message: {message} from Alice")
    elif 'source_b' in name:
        print(f"Received message: {message} from Bob")
    else:
        print(f"Unknown source for message: {message}")

# BSM and detector code

In [ ]:
def create_meas_ops(visibility=1):
    """Sets the photon beamsplitter POVM measurements.

    We are assuming a lot here, see the Netsquid-Physlayer snippet for more info.

    Parameters
    ----------
    visibility : float, optional
        The visibility of the qubits in the detector.

    """
    mu = np.sqrt(visibility)
    s_plus = (np.sqrt(1 + mu) + np.sqrt(1 - mu)) / (2. * np.sqrt(2))
    s_min = (np.sqrt(1 + mu) - np.sqrt(1 - mu)) / (2. * np.sqrt(2))
    m0 = np.diag([1, 0, 0, 0])
    ma = np.array([[0, 0, 0, 0],
                   [0, s_plus, s_min, 0],
                   [0, s_min, s_plus, 0],
                   [0, 0, 0, np.sqrt(1 + mu * mu) / 2]],
                  dtype=complex)
    mb = np.array([[0, 0, 0, 0],
                   [0, s_plus, -1. * s_min, 0],
                   [0, -1. * s_min, s_plus, 0],
                   [0, 0, 0, np.sqrt(1 + mu * mu) / 2]],
                  dtype=complex)
    m1 = np.diag([0, 0, 0, np.sqrt(1 - mu * mu) / np.sqrt(2)])
    n0 = Operator("n0", m0)
    na = Operator("nA", ma)
    nb = Operator("nB", mb)
    n1 = Operator("n1", m1)
    return [n0, na, nb, n1]

class BSMDetector(QuantumDetector):
    """A component that performs Bell basis measurements.

    Measure two incoming qubits in the Bell basis if they
    arrive within the specified measurement delay.
    Only informs the connections that send a qubit of the measurement result.

    """

    def __init__(self, name, system_delay=0., dead_time=0., models=None,
                 output_meta=None, error_on_fail=False, properties=None):
        super().__init__(name, num_input_ports=2, num_output_ports=2,
                         meas_operators=create_meas_ops(),
                         system_delay=system_delay, dead_time=dead_time,
                         models=models, output_meta=output_meta,
                         error_on_fail=error_on_fail, properties=properties)
        self._sender_ids = []

    def preprocess_inputs(self):
        """Preprocess and capture the qubit metadata

        """
        super().preprocess_inputs()
        for port_name, qubit_list in self._qubits_per_port.items():
            if len(qubit_list) > 0:
                self._sender_ids.append(port_name[3:])

    def inform(self, port_outcomes):
        """Inform the MHP of the measurement result.

        We only send a result to the node that send a qubit.
        If the result is empty we change the result and header.

        Parameters
        ----------
        port_outcomes : dict
            A dictionary with the port names as keys
            and the post-processed measurement outcomes as values

        """
        for port_name, outcomes in port_outcomes.items():
            if len(outcomes) == 0:
                outcomes = ['TIMEOUT']
                header = 'error'
            else:
                header = 'photonoutcome'
            # Extract the ids from the port names (cout...)
            if port_name[4:] in self._sender_ids:
                msg = Message(outcomes, header=header, **self._meta)
                self.ports[port_name].tx_output(msg)

    def finish(self):
        """Clear sender ids after the measurement has finished."""
        super().finish()
        self._sender_ids.clear()

## Defining a QPU
We need a QPU for Alice and Bob in order to store and manipulate qubits in memory. We use this to establish ebits between the two parties.

In [111]:
def create_simple_processor(qpu_name, qbit_count=2, depolar_rate=0):
    """Create a quantum processor with basic operations for teleportation."""
    physical_instructions = [
        PhysicalInstruction(instr.INSTR_INIT, duration=3, parallel=True),
        PhysicalInstruction(instr.INSTR_H, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_X, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_Z, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_CNOT, duration=4, parallel=True, topology=[(0, 1)]),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False)
    ]
    memory_noise_model = DepolarNoiseModel(depolar_rate=depolar_rate)
    processor = QuantumProcessor(qpu_name, num_positions=qbit_count, memory_noise_models=[memory_noise_model]*2, 
                                 phys_instructions=physical_instructions)
    return processor

class FSOSwitch(Component):
    def __init__(self, name):
        ports = ["qin0", "qin1", "qin2", "qout0", "qout1"]
        super().__init__(name, port_names=ports)

        # Configuration of the switch parameters
        model_parameters = {
            "short": {
                "init_loss": 0,
                "len_loss": 0.25,
                "init_depolar": 0,
                "len_depolar": 0,
                "channel_len": 1
            },
            "mid": {
                "init_loss": 0,
                "len_loss": 0.25,
                "init_depolar": 0,
                "len_depolar": 0,
                "channel_len": 1.2
            }
        }

        # Model the fibre loss, delay and dephasing of the different routes in the switch
        model_map_short = {
            "delay_model": FibreDelayModel(p_depol_init=model_parameters["short"]["init_depolar"],
                    p_depol_length=model_parameters["short"]["len_depolar"]),
            "quantum_noise_model": FibreDepolarizeModel(),
            "quantum_loss_model": FibreLossModel(p_loss_init=model_parameters["short"]["init_loss"],
                p_loss_length=model_parameters["short"]["len_loss"], rng=None)
        }
        model_map_mid = {
            "delay_model": FibreDelayModel(p_depol_init=model_parameters["mid"]["init_depolar"],
                    p_depol_length=model_parameters["mid"]["len_depolar"]),
            "quantum_noise_model": FibreDepolarizeModel(),
            "quantum_loss_model": FibreLossModel(p_loss_init=model_parameters["mid"]["init_loss"],
                p_loss_length=model_parameters["mid"]["len_loss"], rng=None)
        }

        # Model the three different routes qubits can take through the switch
        qchannel_00 = QuantumChannel(name="qchannel_00", models=model_map_short, length=model_parameters["short"]["channel_len"])
        qchannel_11 = QuantumChannel(name="qchannel_11", models=model_map_short, length=model_parameters["short"]["channel_len"])
        qchannel_21 = QuantumChannel(name="qchannel_21", models=model_map_mid, length=model_parameters["mid"]["channel_len"])
        self.add_subcomponent(qchannel_00)
        self.add_subcomponent(qchannel_11)
        self.add_subcomponent(qchannel_21)

        # Forward inputs and outputs through switch loss channels
        self.ports['qin0'].forward_input(qchannel_00.ports['send'])
        self.ports['qin1'].forward_input(qchannel_11.ports['send'])
        self.ports['qin2'].forward_input(qchannel_21.ports['send'])
        qchannel_00.ports['recv'].forward_output(self.ports['qout0'])
        qchannel_11.ports['recv'].forward_output(self.ports['qout1'])
        qchannel_21.ports['recv'].forward_output(self.ports['qout1'])

In [112]:
# Reset simulation
ns.sim_reset()

# Create QPUs
qpu_a = create_simple_processor("qpu_a")
qpu_b = create_simple_processor("qpu_b")

# Create loss, error and delay models for the quantum fibre channel
photon_loss_init = 0 # Default: 0.2
photon_loss_len = 0.25 # Default: 0.25
photon_depol_init = 0 # Default: 0.009
photon_depol_len = 0 # Default: 0.025
channel_length_a = 1
channel_length_b = 1.2
model_map = {
    "delay_model": FibreDelayModel(p_depol_init=photon_depol_init, p_depol_length=photon_depol_len),
    "quantum_noise_model": FibreDepolarizeModel(),
    "quantum_loss_model": FibreLossModel(p_loss_init=photon_loss_init, p_loss_length=photon_loss_len, rng=None)
}

qchannel_a = QuantumChannel(name="qchannel_a", models=model_map, length=channel_length_a)
qchannel_b = QuantumChannel(name="qchannel_b", models=model_map, length=channel_length_b)

# Setup qubit sources
SS_2_QUBITS = StateSampler(b00)
ext = SourceStatus.EXTERNAL
source_a = QSource(name="source_a", status=ext, state_sampler=SS_2_QUBITS, num_ports=2)
source_b = QSource(name="source_b", status=ext, state_sampler=SS_2_QUBITS, num_ports=2)

# Connect source output to QPU and QChannel ports (deposit one qubit in memory and send the other)
source_a.ports["qout0"].connect(qpu_a.ports['qin0'])
source_a.ports["qout1"].connect(qchannel_a.ports["send"])
source_b.ports["qout0"].connect(qpu_b.ports['qin0'])
source_b.ports["qout1"].connect(qchannel_b.ports["send"])

# Setup msg logging on the quantum channel output
#qchannel_a.ports['recv'].bind_output_handler(debug_output)
# Manual trigger
trigger_a = Clock("single_trigger_a", frequency=1e9, max_ticks=1)
trigger_a.ports["cout"].connect(source_a.ports["trigger"])
trigger_a.start()
trigger_b = Clock("single_trigger_b", frequency=1e9, max_ticks=1)
trigger_b.ports["cout"].connect(source_b.ports["trigger"])
trigger_b.start()

## Time to model the switch

TODO:
- Model Alice and Bob as nodes
- Model switch as node with QPU which returns results to alice and bob
- We need a routing table on the switch input which returns the messages to the appropriate source for qubit corrections

In [ ]:
c = 
# Instantiate Charlie's processor
qpu_c = create_simple_processor("qpu_c", 2, 0)

# Bind the channels to Charlie’s processor
qchannel_a.ports["recv"].connect(qpu_c.ports["qin0"])
qchannel_b.ports["recv"].connect(qpu_c.ports["qin1"])

bsm_lock = False

In [113]:
# Instantiate Charlie's processor
qpu_c = create_simple_processor("qpu_c", 2, 0)

# Bind the channels to Charlie’s processor
qchannel_a.ports["recv"].connect(qpu_c.ports["qin0"])
qchannel_b.ports["recv"].connect(qpu_c.ports["qin1"])

bsm_lock = False

# Function to handle qubit reception and directly perform BSM using qapi
def handle_qubit_reception(qpu):
    global bsm_lock

    # Check if both memory positions are occupied and lock is not active
    if not bsm_lock and not qpu.mem_positions[0].is_empty and not qpu.mem_positions[1].is_empty:
        
        # Set the lock to prevent additional BSM execution
        bsm_lock = True
        print("Lock set. Performing BSM directly with qapi...")
    def program(self):
        # Specify the qubit indices
        q0, q1 = self.get_qubit_indices(2)
        print(f"Inside BSM program: qubit indices {q0}, {q1}")

        # Apply the entanglement swap sequence: CNOT, Hadamard, and measurement
        print(f"STARTING TO FUCKING CPU INSTRUCTIONS: {ns.sim_time()}")
        self.apply(instr.INSTR_CNOT, [q0, q1])
        self.apply(instr.INSTR_H, q0)
        self.apply(instr.INSTR_MEASURE, q0, output_key="m0")
        self.apply(instr.INSTR_MEASURE, q1, output_key="m1")
        print(f"DONE EXECUTING SHIT, YIELDING NOW: {ns.sim_time()}, OBVIOUSLY ITS FUCKING EMPTY: {self.output}")
        yield self.run()
        print(f"AFTER THE FUCKING YIELD: {ns.sim_time()},FUCKING SHIT: {self.output}")

        # Perform measurements directly and get results
        result_m0, _ = qapi.measure(qubit0, observable=ns.Z)
        result_m1, _ = qapi.measure(qubit1, observable=ns.Z)

        # Process and emit results immediately
        results = {"m0": result_m0, "m1": result_m1}
        print(f"Direct Bell State Measurement Results using qapi: {results}")

        # Release the lock after completing the BSM
        bsm_lock = False
        print("Lock released. Ready for next BSM.")
    else:
        print("Either lock is active or one qubit is missing. Waiting for both qubits.")

# Updated input handler for each port to store the qubit in Charlie's memory
def qubit_input_handler(message, qpu, position):
    global bsm_lock

    if not bsm_lock:
        # Place the qubit from the message into the specified memory position
        qpu.mem_positions[position].set_qubit(message.items[0])
        handle_qubit_reception(qpu)
    

# Bind each port to the input handler with specified memory positions
qpu_c.ports["qin0"].bind_input_handler(lambda msg: qubit_input_handler(msg, qpu_c, 0))
qpu_c.ports["qin1"].bind_input_handler(lambda msg: qubit_input_handler(msg, qpu_c, 1))

## Run the sim

In [114]:
# Check that memory is empty at position 0
stats = ns.sim_run() # Run simulation

Either lock is active or one qubit is missing. Waiting for both qubits.
Lock set. Performing BSM directly with qapi...
Direct Bell State Measurement Results using qapi: {'m0': 0, 'm1': 0}
Lock released. Ready for next BSM.
